In [6]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
sys.path.append('../')
from dataset.PointCloudDataset import PointCloudDataset
from dataset.voxelDataset import VoxelDataset

import os, glob

import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
dataset_train  = PointCloudDataset('../dataset/modelnet40_normal_resampled', 
                                                train=True, 
                                                ndata=4000,
                                                file_extension='.txt', 
                                                npoints=4000
                                            )

Loaded 4000 point clouds from 40 classes


In [8]:
data = dataset_train[0]

In [9]:

DEVICE = 'cpu'
from FoldingNet import FoldNet, Encoder, Decoder
checkpoints_path = '../checkpoints/foldingnet'
model = FoldNet(num_points=4000).to(DEVICE)

# load model if exists
models_saved = glob.glob(os.path.join(checkpoints_path, 'model_*.pth'))
if len(models_saved) > 0:
    # get most recent model
    epoches_done = max([int(m.split('_')[-1].split('.')[0]) for m in models_saved])
    model_path = os.path.join(checkpoints_path, f'model_{epoches_done}.pth')
    print(f"Loading model from {model_path}")
    model.load_state_dict(torch.load(model_path))

Loading model from ../checkpoints/foldingnet/model_128.pth


In [10]:
point_cloud, label = data

point_cloud = point_cloud.unsqueeze(0)
point_cloud = point_cloud.to(DEVICE)

decoded, encoded = model(point_cloud)
decoded = decoded.squeeze(0).detach().cpu().numpy()
point_cloud = point_cloud.squeeze(0).detach().cpu().numpy()

In [12]:
#open3d visualization

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)
o3d.visualization.draw_geometries([pcd])

pcd_dec = o3d.geometry.PointCloud()
pcd_dec.points = o3d.utility.Vector3dVector(decoded)
o3d.visualization.draw_geometries([pcd_dec])


: 